In [1]:
import os
import data_pre
import input_data
import model_run
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
df = input_data.get_data ('train_data/train.csv')
test_df = pd.read_csv('dev.csv')
df['is_humor'].value_counts(normalize = True)
test_df['is_humor'].value_counts(normalize = True)
df['humor_controversy'] = (df['humor_controversy'].fillna(0)).astype(int)
test_df['humor_controversy'] = (test_df['humor_controversy'].fillna(0)).astype(int)


In [2]:
X_train = df[['text']]
y_train = df[['is_humor','humor_rating','humor_controversy','offense_rating']]
X_test = test_df[['text']]
y_test = test_df[['is_humor','humor_rating','humor_controversy','offense_rating']]

In [3]:
# Break data
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)
train_humor,train_humor_rating,train_humor_contro,train_offense_rating = input_data.break_df(y_train)
test_humor,test_humor_rating,test_humor_contro,test_offense_rating = input_data.break_df(y_test)

In [4]:
#Pre-Processing
final_text_ = data_pre.data_processing ( X_train[['text']] )
final_x_test_= data_pre.data_processing ( X_test[['text']] )

In [5]:
vectorizer = TfidfVectorizer(max_features = 8000,ngram_range=(1,3))
final_text = vectorizer.fit_transform(final_text_)
final_x_test = vectorizer.fit_transform(final_x_test_)#vocab = vectorizer.get_feature_names()


In [6]:
final_text

<8000x8000 sparse matrix of type '<class 'numpy.float64'>'
	with 79784 stored elements in Compressed Sparse Row format>

In [7]:
vocab = vectorizer.get_feature_names()
#print(vocab)

In [8]:
import numpy as np
dist = np.sum(final_text, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

[[1.023041   1.36756974 1.43924177 ... 1.03677139 2.94494703 0.81429832]] aaron


In [9]:
from sklearn.svm import SVR,SVC
#df_run_2 = model_run.run_model(final_text,humor_contro,final_x_test,y_test[['humor_controversy']])

In [14]:
test_df['text'] = test_df['text'].str.replace("[^a-zA-Z]", " ")
tokenized_doc = test_df['text'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
detokenized_doc = [] 
for i in range(len(test_df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
test_df['text'] = detokenized_doc

In [ ]:
#comparision with BERT

bert_df = pd.read_csv('public_dev_bert(2).csv')
f1_score( test_df['is_humor'],bert_df[['is_humor']])#,confusion_matrix(y_test['is_humor'],bert_df[['is_humor']])

In [ ]:
bert_df
test_df['berthumor'] = bert_df['is_humor']
test_df

In [ ]:
f1_score( test_humor,bert_df['is_humor']),precision_score(test_humor,bert_df['is_humor']),confusion_matrix(test_humor,bert_df['is_humor'])

In [ ]:
test = pd.DataFrame()
#test['Actual'] = test_df['is_humor']
#test['Bert'] = bert_df['is_humor']
#test['NB'] = preds

In [ ]:
#test = pd.DataFrame()
#test['Actual'] = new_X_test['humor_controversy']
#test['Bert'] = bert_df['humor_controversy']
#test['NB'] = preds
#new_X_test.isnull().values.any(),test['Bert'].isnull().values.any(),new_X_test.shape,test.shape,bert_df.shape

In [ ]:
test['try1'] = np.where((test['Bert'] > 0) & (test['NB'] < 1 )
                     ,0,test['Bert'])
test.head()
test.isnull().values.any()

#provides a slight improvement
f1_score( test_humor,test['try1']),precision_score(test_humor,test['is_humor'])confusion_matrix(test_humor,test['try1'])

In [ ]:
#f1_score( new_X_test[['humor_controversy']],test['try1']),confusion_matrix(new_X_test[['humor_controversy']],test['try1'])

In [ ]:
print (test.shape)
new_X_test['humor_controversy'].to_csv('testtry.csv')

In [ ]:
preds.shape

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
f1_score( test_humor,y_pred),precision_score( test_humor,y_pred),confusion_matrix( test_humor,y_pred)

In [ ]:
vect = CountVectorizer(ngram_range=(1,4)).fit(df['text'])
X_train_vectorized = vect.transform(df['text'])
model = SVR(kernel = 'linear',tol = 0.001,C=1)
#Changing Scale from 0 to 5 -> 0 to 500
df['offense_rating'] = 100 * df['offense_rating']
model.fit(X_train_vectorized,df['offense_rating'])
y_predicted= model.predict(vect.transform(test_df['text']))
y_predicted[y_predicted<0]=0
y_predicted[y_predicted>500]=499
#df[['humor_rating_predicted']]=y_predicted_svr
y_preds = np.around(y_predicted/100,decimals=2) 
test_df['svroff'] = y_preds

from sklearn.metrics import mean_squared_error
mean_squared_error(test_df['offense_rating'],test_df['svroff'], squared=False),test_df

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor()
#y_train['humor_rating'] = 100 * y_train['humor_rating'] 
reg.fit(X_train_vectorized,df['offense_rating'])
gbrpred = reg.predict(vect.transform(test_df['text']))
#gbrpred = np.around(gbrpred,decimals=2)
gbrpred[gbrpred<10]=0
gbrpred[gbrpred>500]=499
test_df['gbrpreds'] = gbrpred
test_df['gbrpreds'] = np.around(test_df['gbrpreds']/100,decimals=2)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_df['offense_rating'],test_df['gbrpreds'], squared=False),test_df

In [ ]:
#Filtering Data

X_train = df.loc[lambda x: x['is_humor'] == 1]
X_test = test_df.loc[lambda x: x['berthumor'] == 1]
y_train = df.loc[lambda x: x['is_humor'] == 1]
y_test = test_df.loc[lambda x: x['berthumor'] == 1]
y_rest = test_df.loc[lambda x: x['berthumor'] == 0]
vect = CountVectorizer(ngram_range=(1,4)).fit(X_train['text'])
X_train_vectorized = vect.transform(X_train['text'])

model = SVR(kernel = 'linear',tol = 0.001,C=5)
#train_humor_rating['humor_rating'] = (train_humor_rating['humor_rating'].fillna(0)).astype(float)
#test_humor_rating['humor_rating'] = (test_humor_rating['humor_rating'].fillna(0)).astype(float)
y_train['humor_rating'] = 100 * y_train['humor_rating']
model.fit(X_train_vectorized,y_train['humor_rating'])
y_predicted= model.predict(vect.transform(X_test['text']))
y_predicted[y_predicted<0]=0
y_predicted[y_predicted>500]=499
#df[['humor_rating_predicted']]=y_predicted_svr
y_preds = np.around(y_predicted/100,decimals=2) 
y_test['svrpreds'] = y_preds
y_test

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor()
#y_train['humor_rating'] = 100 * y_train['humor_rating'] 
reg.fit(X_train_vectorized,y_train['humor_rating'])
gbrpred = reg.predict(vect.transform(X_test['text']))
#gbrpred = np.around(gbrpred,decimals=2) 
y_test['gbrpreds'] = gbrpred
y_test['gbrpreds'] = np.around(y_test['gbrpreds']/100,decimals=2)

y_train,y_test

In [ ]:
#y_predicted
#y_preds = np.around(y_predicted,decimals=2) 
#y_preds
y = pd.concat([y_test, y_rest], ignore_index = False)
y=y.sort_values(by=['id'])
#print (y)
#y.to_csv('check.csv')
y = y.fillna(0)
#y
y_tt = y.loc[lambda x: x['berthumor'] == 1]
#y_tt

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
#f1_score( test_humor_contro,y_predicted),precision_score( test_humor_contro,y_predicted)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_tt['humor_rating'],y_tt['svrpreds'], squared=False),mean_squared_error(y_tt['humor_rating'],y_tt['gbrpreds'], squared=False)

In [ ]:
model.get_params()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_1= RandomForestRegressor(max_depth=10)
model_1.fit(final_text,train_offense_rating[['offense_rating']])
y_predicted_RFR= model_1.predict(final_x_test)
y_preds_RFR = np.around(y_predicted_RFR,decimals=2) 
mean_squared_error(test_offense_rating,y_preds_RFR, squared=False)
y_preds_RFR

In [ ]:
#test_df['predicted'] = y_predicted


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#for x in [40,50,60,70,80,90,100,110,120,130,140]:
clf = GradientBoostingClassifier(n_estimators=40, learning_rate=2,max_depth=1, random_state=0).fit(train_humor_rating[['humor_rating']],train_humor_contro[['humor_controversy']])
#clf.score(X_test, y_test)
#y_predicted = clf.predict(y_predicted_svr)
#print (f1_score( test_humor_contro['humor_controversy'],y_predicted),precision_score( test_humor_contro['humor_controversy'],y_predicted))

In [ ]:
#X_train, y_train, = (df[['id','text']],df[['is_humor','humor_rating','humor_controversy','offense_rating']])

In [ ]:
#test_df[['humor_controversy']]=y_predicted.astype(int)

In [ ]:
#test_df[['humor_rating']]=round(test_df[['humor_rating']],3)

In [ ]:
from xgboost import XGBRegressor
XGBModel = XGBRegressor(booster = 'dart')
XGBModel.fit(final_text,train_offense_rating[['offense_rating']] , verbose=False)

# Get the mean absolute error on the validation data :
XGBpredictions = XGBModel.predict(final_x_test)
mean_squared_error(test_offense_rating,XGBpredictions, squared=False)

In [ ]:
proverb_df = pd.read_pickle('datasets/proverbs.pickle')
proverb['text']= pd.DataFrame(proverb_df)

In [ ]:
proverb

In [ ]:
source[['text']] = pickled_df[['text']]
source['is_humor'] = 0

In [ ]:
unpickled_df = pd.read_pickle('datasets/humorous_oneliners.pickle')
pickled_df[['text']] = pd.DataFrame(unpickled_df)

In [ ]:
source[['text']] = pickled_df[['text']]
source['is_humor'] = 0

In [ ]:
#Checking for other dependencies in dataset

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
#vect = CountVectorizer(ngram_range=(1,4)).fit(X_train['text'])
#final_text
vect = CountVectorizer(ngram_range=(2,4)).fit(X_train['text'])
X_train_vectorized = vect.transform(X_train['text'])
#X_train_vectorized = vect.transform(X_train['text'])
clfrNB2 = MultinomialNB()
#clfrNB = GaussianNB()

clfrNB2.fit([X_train_vectorized,train_humor[['is_humor']]],train_humor_contro[['humor_controversy']])
#clfrNB.fit(X_train_vectorized,train_humor[['is_humor']])
preds2 = clfrNB2.predict(vect.transform(X_test['text']))



In [ ]:
f1_score( test_humor_contro,preds2),precision_score( test_humor_contro,preds2),confusion_matrix( test_humor_contro,preds2)